In [ ]:
import os
import logging
import numpy as np

from automata.singletons.dependency_factory import dependency_factory, DependencyFactory
from automata.singletons.py_module_loader import py_module_loader
from automata.symbol.graph import SymbolGraph
from automata.symbol_embedding.vector_databases import (
    ChromaSymbolEmbeddingVectorDatabase,
)
from automata.symbol_embedding import SymbolCodeEmbedding
from automata.llm.providers.openai_llm import OpenAIEmbeddingProvider



In [ ]:
py_module_loader.reset()
dependency_factory.reset()
project_name = "automata"
py_module_loader.initialize(f"..", project_name) # ! Can replace with your own project


In [ ]:
symbol_graph = SymbolGraph(
    os.path.join(DependencyFactory.DEFAULT_SCIP_FPATH, f"{project_name}.scip")
)

code_embedding_db = ChromaSymbolEmbeddingVectorDatabase(
    project_name,
    persist_directory=DependencyFactory.DEFAULT_CODE_EMBEDDING_FPATH,
    factory=SymbolCodeEmbedding.from_args,
)

dependency_factory.set_overrides(
    **{
        "symbol_graph": symbol_graph,
        "code_embedding_db": code_embedding_db,
    }
)

symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get("symbol_code_embedding_handler")
embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')

code_embeddings = symbol_code_embedding_handler.get_all_ordered_embeddings()

code_embeddings_array = np.array([np.array(ele.vector) for ele in code_embeddings])

query = "How do I build an agent?"

print(f"Demonstrating embedding Similarity Search search results for query = '{query}'")
code_similarity_results = embedding_similarity_calculator.calculate_query_similarity_dict(code_embeddings, query)
for i,rank in enumerate(list(code_similarity_results.items())[:10]):
    print(f"rank {i} = {rank[0].full_dotpath} with distance {rank[1]:.3f}")

